In [1]:
# importing libraries 

import hashlib
from imageio import imread 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import numpy as np

In [2]:
# sklearn has lbp enable feature

from skimage import feature

In [3]:
import time
import concurrent.futures


In [4]:
# parallel time
start = time.perf_counter()

In [5]:
def lbp_features(img , radius = 1, sampling_pixels=8):
    
    # LBP operates in single channel images so if RGB images are provided
    # we have to convert it to grayscale
    if (len(img.shape) >2):
        img = img.astype(float)
        # RGB to grayscale convertion using Luminance
        img = img[:,:,0]*0.3 + img[:,:,1]*0.59 + img[:,:,2]*0.11
        
    # converting to unside_int8 (uint8) type for 256 graylevels
    img = img.astype(np.uint8)
    
    # normalize values can also help improving description 
    i_min = np.min(img)
    i_max = np.max(img)
    if (i_max -i_min !=0):
        img = (img - i_min)/(i_max - i_min)
        
    #compute LBP
    lbp = feature.local_binary_pattern(img, sampling_pixels, radius, method = "uniform")
    
    # LBP returns a matrix with the codes, so we compute the histogram
    (hist, _) = np.histogram(lbp.ravel(), bins = np.arange(0, sampling_pixels +3), range=(0,255))
    
    #normalization
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    #return the histogram of local binary patterns
    return hist


def Euclidean_distance(p,q):
    dist = np.sqrt(np.sum(np.square(p-q)))
    return dist
    

In [6]:
import os 
os.getcwd()

'C:\\Users\\Agupta\\Downloads\\duplicate image detection'

In [7]:
os.chdir(r'..\\100_Images_Lab')

In [8]:
files_list=os.listdir()
len(files_list)

107

In [9]:
img = []
lbp_n =[]
dQ_H = []
dup = []

In [10]:
import hashlib, os 
import imageio

for index, filename in enumerate(os.listdir('.')):
    if os.path.isfile(filename):
        with open(filename,'rb') as f:
            img.append(imageio.imread(f.read()))            

In [11]:
import hashlib, os 
import imageio
duplicates = []
hash_keys =dict()
hash_code=[]
for index in range(len(img)):
    image_num = hashlib.md5(img[index]).hexdigest()
    hash_code.append(image_num)
    if image_num not in hash_keys:
        hash_keys[image_num] = index
    else :
        duplicates.append((index,hash_keys[image_num]))

In [12]:
duplicates

[(28, 27), (41, 40), (68, 67), (80, 79), (84, 83), (87, 86), (99, 98)]

In [13]:
def detectDup(img):
    for i in range (len(img)):
        lbp_n.append(lbp_features(img[i],2,8))
        
    # compute euclidean distance
    for i in range (len(img)):
        for j in range (len(img)):
            dQ_H.append(Euclidean_distance(lbp_n[i],lbp_n[j]))

    for i in range (len(img)):
        for j in range (len(img)):
            if dQ_H[i] == dQ_H[j] and hash_code[i]==hash_code[j] and i != j:
                dup.append((i,j)) 
                
    for file_indexes in duplicates[:30]:
        try:
            plt.subplot(121),plt.imshow(imread(files_list[file_indexes[1]]))
            plt.title(str(file_indexes[1])+' original file'), plt.xticks([]), plt.yticks([])

            plt.subplot(122),plt.imshow(imread(files_list[file_indexes[0]]))
            plt.title(str(file_indexes[0])+' duplicate file'), plt.xticks([]), plt.yticks([])
            plt.show()

        except OSError as e:
            continue      
            
        img1=imageio.imread('..\\100_Images_Lab\\1.jpg')
        img2=imageio.imread('..\\100_Images_Lab\\80 - Copy.jpg')
        img3=imageio.imread('..\\100_Images_Lab\\80.jpg')
        img4=imageio.imread('..\\100_Images_Lab\\31.jpg')
        img5=imageio.imread('..\\100_Images_Lab\\32.jpg')
        img6=imageio.imread('..\\100_Images_Lab\\4.jpg')

        lbp1=lbp_features(img1,2,8)
        lbp2=lbp_features(img2,2,8)
        lbp3=lbp_features(img3,2,8)
        lbp4=lbp_features(img4,2,8)
        lbp5=lbp_features(img5,2,8)
        lbp6=lbp_features(img6,2,8)

        dQ1_H = Euclidean_distance(lbp1,lbp2)
        dQ2_H = Euclidean_distance(lbp1,lbp3)
        dQ3_H = Euclidean_distance(lbp1,lbp4)
        dQ4_H = Euclidean_distance(lbp1,lbp6)
        dQ5_H = Euclidean_distance(lbp1,lbp6)
        dQ6_H = Euclidean_distance(lbp1,lbp1)
        

        plt.figure(figsize =(10,8))
        plt.subplot(231); plt.imshow(img1) ; plt.title('ORIGINAL')
        plt.subplot(232); plt.imshow(img2) ; plt.title('rank 1: %.4f' %dQ1_H)
        plt.subplot(233); plt.imshow(img3) ; plt.title('rank 2: %.4f' %dQ2_H)
        plt.subplot(234); plt.imshow(img4) ; plt.title('rank 3: %.4f' %dQ3_H)
        plt.subplot(235); plt.imshow(img5) ; plt.title('rank 4: %.4f' %dQ4_H)
        plt.subplot(236); plt.imshow(img5) ; plt.title('rank 5: %.4f' %dQ5_H)
        
        vals = range(len(lbp1))
        plt.figure(figsize =(10,5))
        plt.subplot(231); plt.bar(vals,lbp1);
        plt.title('ORIGINAL'); plt.axis('off')

        plt.subplot(232); plt.bar(vals,lbp2);
        plt.title('rank 1'); plt.axis('off')

        plt.subplot(233); plt.bar(vals,lbp3);
        plt.title('rank 2'); plt.axis('off')

        plt.subplot(234); plt.bar(vals,lbp4);
        plt.title('rank 3'); plt.axis('off')

        plt.subplot(235); plt.bar(vals,lbp5);
        plt.title('rank 4'); plt.axis('off')

        plt.subplot(236); plt.bar(vals,lbp6);
        plt.title('rank 5'); plt.axis('off')
        


In [14]:
#print(dQ_H[0],dQ_H[54],dQ_H[55])

In [15]:
# hash_keys.values()

In [16]:
# print(len(dup))
# print(dup)

In [17]:
with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(detectDup,img)

In [18]:
# dQ7_H = Euclidean_distance(lbp2,lbp1)
# dQ7_H

In [19]:
files_list=os.listdir()
len(files_list)

107

In [20]:
end = time.perf_counter()

In [21]:
print(f'Parallel time = {round(end-start,2)} seconds')

Parallel time = 1.27 seconds


In [22]:
# delete duplicate files 
""""
for index in duplicates:
    os.remove(files_list[index[0]]) """

'"\nfor index in duplicates:\n    os.remove(files_list[index[0]]) '